### Initialization

In [1]:
import glob
import json
import re
from nltk import tokenize
import textdistance
import random
import string
import os
import numpy
import pandas
import fasttext
import gensim
import conllu
import editdistance
import udapi

In [2]:
from utils.text_extraction import *
from utils.document_processing import *
from utils.subject_extraction import *
from utils.similarity import *
from utils.context_extraction import *

In [3]:
import psycopg2
psycopg2_conn = psycopg2.connect(dbname='public_contracts', user='postgres', password='admin', host='localhost', port='5432')

In [4]:
from ufal.udpipe import *
# load model from the given file;
# if the file does not exist, expect a Segmentation fault
udpipe_model = Model.load("../model/udpipe/udpipe-ud-2.4-190531/czech-pdt-ud-2.4-190531.udpipe")

In [5]:
# create a UDPipe processing pipeline with the loaded model,
# with "horizontal" input (a sentence with space-separated tokens),
# default setting for tagger and parser,
# and CoNLL-U output
udp_pipeline = Pipeline(udpipe_model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")

### Documents 

In [6]:
from utils.document_processing import *

In [7]:
loader = DatabaseDocumentLoader(psycopg2_conn)
loader.load_documents()
documents = loader.prepare_documents()

Running query: select * from document where processed=True
Preparing total 1008 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%


In [8]:
vzdirs = '../../test-data/*/*/*/*'
ignore_contracts = ['2096','sportovni-areal-zs-slatina-1-etapa_25534',
                    'revitalizace-lokality-spaliste-k-u-stare-mesto-u-uherskeho-hradiste-akceptacni-cislo-12136894-cz-1-02-4-2-00-12-16399-dodatecne-prace_9496',
                    '35226', 'kompostujeme-v-obci-sebranice-dodavka-stepkovace-a-komposteru', 'zvysovani-povedomi-verejnosti-o-biodiverzite-luk-a-pastvin',
                    'nakup-noveho-uzitkoveho-elektromobilu-typu-bev-vozidlo-s-bateriovym-pohonem']

ref_loader = ReferenceDocumentLoader(vzdirs)
# ref_loader.extract_text_from_documents(0,1024)
ref_loader.load_documents_from_extracts(ignore_contracts)
ref_documents = ref_loader.prepare_documents()

Loading total 339 extracts
Preparing total 339 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 39.0%
Progress: 49.0%
Progress: 59.0%
Progress: 69.0%
Progress: 78.0%
Progress: 88.0%
Progress: 98.0%


In [9]:
ignore_mask = count_occurence_vector('qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNMáčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ0123456789')
ignore_mask[0]=0
matcher = DocumentMatcher(ref_documents, documents)
matcher.count_most_similar_documents(ignore_mask=ignore_mask)
matcher.aggregate_documents()
df_contract = matcher.filter_aggregated()

Counting most similar documents for total 339 reference documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 39.0%
Progress: 49.0%
Progress: 59.0%
Progress: 69.0%
Progress: 78.0%
Progress: 88.0%
Progress: 98.0%


### Subject context

In [10]:
from utils.context_extraction import *

In [11]:
contracts = []
for contr_name, docs in df_contract.groupby('contr_name'):
    contr_text = ''
    contr_ids = []
    for index, doc in docs.iterrows():
        doc_name = doc['doc_name']
        contr_text += '======================='+doc_name+'=======================\n'
        contr_text += doc['doc_text']
        contr_ids.append(doc['contr_id'])
    contr_id = get_most_frequent(contr_ids)
    contr = {'id': contr_id, 'contr_name': contr_name, 'text': contr_text}
    contracts.append(contr)

In [12]:
DEF_KEYWORDS = {
    'Předmět smlouvy': 10,
    'Předmět díla': 10,
    'Předmět plnění': 10,
    'Předmět veřejné zakázky': 10,
    'Vymezení předmětu': 10,
    'Vymezení plnění': 10,
    'Popis předmětu': 10,
    'Název veřejné zakázky': 3,
    'Veřejná zakázka': 1,
    'Veřejné zakázce': 1,
    'Předmět': 1,
    'Popis': 1
}

extractor = AdvancedSubjectContextExtractor(keywords=DEF_KEYWORDS, subj_range=2040)
for contr in contracts:
    contr_text = contr['text']
    contr['subj_context'] = extractor.get_subject_contexts(contr_text)

In [13]:
df_contracts = pandas.DataFrame(contracts)
# df_contracts

### Subject extraction

In [14]:
from utils.subject_extraction import *

In [15]:
from utils.subject_context_preprocessing import *

In [16]:
from utils.conllu_preprocessing import *

### EvaluationMachine

In [17]:
class EvaluationMachine:
    
    _core = None
    _subjects = None
    _ref_subjects = None
    _similarity_matrix = None
    
    def __init__(self, core=None):
        self._core = core
    
    def evaluateSubjects(self, subjects, ref_subjects):
        self._subjects = [' '.join(s) if isinstance(s, list) else s for s in subjects]
        self._ref_subjects = [' '.join(s) if isinstance(s, list) else s for s in ref_subjects]
        return self.evaluate()
    
    def evaluate(self):
        self.compute_matrix()
        return self._similarity_matrix
        
    def compute_matrix(self):
        sl = len(self._subjects)
        rl = len(self._ref_subjects)
        result_matrix = numpy.zeros((sl, rl))
        for i in range(sl):
            for j in range(rl):
                d = self._core.compute(self._subjects[i], self._ref_subjects[j])
                result_matrix[i][j] = d
        self._similarity_matrix = result_matrix
        
    def display(self, with_headers=False):
        if with_headers:
            df = pandas.DataFrame(self._similarity_matrix, columns=self._ref_subjects, index=self._subjects)
        else:
            df = pandas.DataFrame(self._similarity_matrix)
        display(df)
        
    
class RandomEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(RandomSimilarityMachine())
        
class JaccardEvaluationMachine(EvaluationMachine):

    def __init__(self, core=None):
        super().__init__(JaccardSimilarityMachine())
        
class FastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(FastTextSimilarityMachine(core))
        
class GensimFastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(GensimFastTextSimilarityMachine(core))

In [18]:
class SubjectContextCopyExtractor(SubjectExtractor):
    
    def __init__(self, param=None):
        pass
        
    def extract(self, text):
        return text

### Similarity functions

In [19]:
from utils.similarity import *

### Measurements

In [20]:
from utils.evaluation import *

In [21]:
from udapi.core.document import Document

def apply_transformation(data, transformation):
    if isinstance(data, list):
        return [transformation(text) for text in data]
    return transformation(data)

def create_conllu_document(decomposition):
    doc = Document()
    doc.from_conllu_string(decomposition)
    return doc

def create_conllu_sentence_list(decomposition):
    doc = create_conllu_document(decomposition)
    return [b.trees[0] for b in doc.bundles]

def tag_text(text):
    return udp_pipeline.process(text)

class SubjectSentenceEvaluationMachine:
    
    _df_contracts = None
    _subj_context_preprocessor = None
    
    def __init__(self, df_contracts, subj_context_preprocessor=SubjectContextPreprocessor()):
        self._df_contracts = df_contracts
        self._subj_context_extractor = subj_context_preprocessor
        
    def preprocess(self):
        self._df_contracts['filtered_context'] = self._df_contracts['subj_context'].apply(\
            lambda text: self._subj_context_extractor.process(text))
#         self._df_contracts['subj_context_decomposition'] = self._df_contracts['filtered_context'].apply(tag_data)
#             return self._df_contracts
    
    def process(self):
        self._df_contracts = validate_subjects(self._df_contracts, 'filtered_context')
        return self._df_contracts

    def evaluate(self):
        return self._df_contracts['valid_score'].mean()*100
    
    def evaluateSentenceNum(self):
        self._df_contracts['sent_cnt_diff'] = \
            self._df_contracts['subj_context_decomposition'].apply(lambda decomposition: len(create_conllu_sentence_list(decomposition))) \
            - self._df_contracts['subj_context_decomposition_ref'].apply(lambda decomposition: len(create_conllu_sentence_list(decomposition)))
        return self._df_contracts['sent_cnt_diff'].abs().mean()

In [22]:
%%time

transformers = [
                NumeralLinesFilter(too_many_numerals_ratio_threshold=0.5),
                TooShortLinesFilter(too_short_line_threshold=5),
                IrrelevantLinesFilter(keywords=['strana', 'stránka', 'e-mail'], max_line_length=75, lower=True),
                IrrelevantLinesFilter(keywords=['Tel:', 'Fax:', 'IČ:', 'IČO:', 'DIČ:'], max_line_length=75, lower=False),
                IrrelevantLinesRegexFilter(patterns=[r'www', r'[\w\-\.]+\s*@\s*([\w\-]+\.)+[\w\-]{2,4}']),  # email
                IrrelevantLinesRegexFilter(patterns=[r'(\+\d{2,3}){0,1}(\s{0,1}\d{3}){3}']),  # phone
                RegexReplaceTransformer(pattern_to_transform=r',([\s]+[A-Z][a-z ])', result_pattern='.\g<1>'),
                RegexReplaceTransformer(pattern_to_transform=r'\n[ \t]*(.{0,1}[\d]+.{0,1})+[ \t]*',  # paragraph numbers
                                        result_pattern='\n'),
                BlankLinesFilter(replacement='\n', top_n_frequency=200, top_n_var_threshold=5,
                                 full_line_threshold=0.85, min_max_line_length=0),
                ReplaceMarksTransformer(marks_to_transform='„“', result_mark='"'),
                TooLongLinesTransformer(forbidden_delimiters='aábcčdďeéěfghiíjklmnňoópqrřsštťuúůvwxyýzž0123456789',
                                        special_delimiters={'-': (r'[\s,\.](-)[\s]+[^(Kč)]', 1)},
                                        too_long_line_treshold=200),
                RegexReplaceTransformer(pattern_to_transform=r'([^\n])[ ]*\n', result_pattern='\g<1>.\n'),
                RegexReplaceTransformer(pattern_to_transform=r'(([Nn]ázev|[Pp]opis)[^\n,.:"()]{5,})(\s[A-Z][^\n:]{10})',
                                        result_pattern='\g<1>:\g<3>'),
                ReplaceMarksTransformer(marks_to_transform=[':'], result_mark=':.'),
                ReplaceMarksTransformer(marks_to_transform=['..'], result_mark='.'),
                RegexReplaceTransformer(pattern_to_transform=r'\([^\n()]*\)', result_pattern=''),
                AddLine(line='\n'),
                QuotedContractNameExtractor(name_tag='<CONTRACT_NAME>;<CONTRACT_NAME/>',
                                            false_keywords=['přílo', 'dále', 'jen'],
                                            positive_keywords=['názvem'],
                                            context_range=30, min_length=25),
                StructuredContractNameExtractor(name_tag='<CONTRACT_NAME>;<CONTRACT_NAME/>',
                                                false_keywords=['přílo', 'dále', 'jen', '"'],
                                                positive_keywords=['název', 'stavb', 'projekt'], min_length=5,
                                                delims=[':.']),
                ItemColonExtractor(item_tag='<ITEM>;<ITEM/>', patterns=[r'(zboží|položk).{,10}:']),
                StructureItemEnumerationExtractor(item_tag='<ITEM>;<ITEM/>', enumeration_pattern=r'010(10)*',
                                                  full_line_length=150, delim=':.'),
                # CharItemEnumerationExtractor(item_tag='<ITEM>;<ITEM/>',
                #                              enumeration_pattern=r'1+',
                #                              forbidden_chars='aábcčdďeéěfghiíjklmnňoópqrřsštťuúůvwxyýzž0123456789'),
                CharTupleItemEnumerationExtractor(item_tag='<ITEM>;<ITEM/>',
                                                  enumeration_pattern=r'1+',
                                                  forbidden_tuples=['Vy', 'Př', 'Za', 'Pr', 'Ob', 'Po']),
                HeaderItemEnumerationExtractor(item_tag='<ITEM>;<ITEM/>', header_pattern='[Pp]ředmět.{0,20}je.{0,5}$'),
]

# df_slice = df_sorted_contracts[:10].copy()
evaluator = SubjectSentenceEvaluationMachine(df_contracts, subj_context_preprocessor=SubjectContextPreprocessor())
evaluator.preprocess()
evaluator.process()
display(evaluator.evaluate())
# display(evaluator.evaluateSentenceNum())

100.0

Wall time: 813 ms


In [23]:
df_contracts = evaluator._df_contracts
df_contracts

,id,contr_name,text,subj_context,filtered_context,valid_score,filtered_context_ref
0,71,e-zakazky/48a575ba-f8fe-47f2-bbb6-c6ddb26390ef...,=======================2017126-115451.pdf=====...,[PŘEDMĚT SMLOUVY\nZhotovitel se zavazuje k pro...,[PŘEDMĚT SMLOUVY.\nZhotovitel se zavazuje k pr...,1,PŘEDMĚT SMLOUVY.\nZhotovitel se zavazuje k pro...
1,72,e-zakazky/73b0be6b-6b49-4ab4-a6a0-508b83a305e1...,=======================Formulář - Informace o ...,[PŘEDMĚT DÍLA\n\n1.1. Předmětem díla je proved...,[PŘEDMĚT DÍLA.\n\nPředmětem díla je provedení ...,1,PŘEDMĚT DÍLA.\n\nPředmětem díla je provedení s...
2,74,e-zakazky/c265fdf8-044f-44e5-b36e-df4ca53c6179...,=======================Dodatečné informace 01....,[Předmět smlouvy\n1. Zhotovitel se touto smlou...,[Předmět smlouvy.\nZhotovitel se touto smlouvo...,1,Předmět smlouvy.\nZhotovitel se touto smlouvou...
3,61,eagri/mze/contract_display_6220,=======================01_výzva k podání nabíd...,[Předmět smlouvy\n\nPředmětem smlouvy je prove...,[Předmět smlouvy.\n\nPředmětem smlouvy je prov...,1,Předmět smlouvy.\n\nPředmětem smlouvy je prove...
4,64,ezak/mpsv/P15V00000447,=======================AKRIS_příloha-01_Kvalif...,[Předmět veřejné zakázky a sjednaná cena ve sm...,[Předmět veřejné zakázky a sjednaná cena ve sm...,1,Předmět veřejné zakázky a sjednaná cena ve sml...
5,66,ezak/mpsv/P16V00000187,=======================Příloha č. 1 SOD_Projek...,"[Předmět smlouvy, předmět díla\n\nPředmětem té...","[Předmět smlouvy, předmět díla.\n\nPředmětem t...",1,"Předmět smlouvy, předmět díla.\n\nPředmětem té..."
6,67,ezak/mpsv/P16V00000240,=======================P1_Výkaz výměr (slepý)....,[Předmět Smlouvy\n2.1. Zhotovitel se zavazuje ...,[Předmět Smlouvy.\nZhotovitel se zavazuje prov...,1,Předmět Smlouvy.\nZhotovitel se zavazuje prové...
7,103,nipez/mo/P18V00009525,=======================VZOR_smlouvy.doc=======...,[Předmět smlouvy\n\n2.1. \nPředmětem smlouvy j...,[Předmět smlouvy.\n\nPředmětem smlouvy je záva...,1,Předmět smlouvy.\n\nPředmětem smlouvy je závaz...
8,207,nipez/mo/P18V00012708,=======================Oznámení o výběru dodav...,[Název veřejné zakázky: Sponkovače a spojovací...,[Název veřejné zakázky:. Sponkovače a spojovac...,1,Název veřejné zakázky:. Sponkovače a spojovací...
9,397,nipez/mo/P18V00019354,=======================PSP s přílohami.zip====...,[Předmět smlouvy\n\n\n1. Poskytovatel se zavaz...,[Předmět smlouvy.\n\nPoskytovatel se zavazuje ...,1,Předmět smlouvy.\n\nPoskytovatel se zavazuje d...


In [24]:
class SubjectItemsEvaluationMachine:
    
    def __init__(self, df_contracts,\
                 subj_context_preprocessor=SubjectContextPreprocessor(),\
                 conllu_context_preprocessor=ConlluSubjectContextPreprocessor()):
        self._df_contracts = df_contracts
        self._subj_context_extractor = subj_context_preprocessor
        self._conllu_context_extractor = conllu_context_preprocessor
        
    def preprocess(self):
        self._df_contracts['filtered_context'] = self._df_contracts['subj_context'].apply(\
            lambda text: self._subj_context_extractor.process(text))
        self._df_contracts['subj_context_decomposition'] = self._df_contracts['filtered_context'].apply(
            lambda data: apply_transformation(data, tag_text))
        self._df_contracts['subj_items'] = self._df_contracts['subj_context_decomposition'].apply(\
            lambda data: apply_transformation(self._conllu_context_extractor.process(
                            apply_transformation(data, create_conllu_document)), str))
        return self._df_contracts
    
    def process(self):
        self._df_contracts = validate_subjects(self._df_contracts, 'subj_items')
        return self._df_contracts

    def evaluate(self):
        return self._df_contracts['valid_score'].mean()*100

In [25]:
%%time

transformers = [
    NumeralLinesFilter(too_many_numerals_ratio_threshold=0.5),
    TooShortLinesFilter(too_short_line_threshold=5),
    IrrelevantLinesFilter(keywords=['strana', 'stránka', 'e-mail'], max_line_length=75, lower=True),
    IrrelevantLinesFilter(keywords=['Tel:', 'Fax:', 'IČ:', 'IČO:', 'DIČ:'], max_line_length=75, lower=False),
    IrrelevantLinesRegexFilter(patterns=[r'www', r'[\w\-\.]+\s*@\s*([\w\-]+\.)+[\w\-]{2,4}']),  # email
    IrrelevantLinesRegexFilter(patterns=[r'(\+\d{2,3}){0,1}(\s{0,1}\d{3}){3}']),  # phone
    RegexReplaceTransformer(pattern_to_transform=r',([\s]+[A-Z][a-z ])', result_pattern='.\g<1>'),
    RegexReplaceTransformer(pattern_to_transform=r'\n[ \t]*(.{0,1}[\d]+.{0,1})+[ \t]*',  # paragraph numbers
                            result_pattern='\n'),
    BlankLinesFilter(replacement='\n', top_n_frequency=200, top_n_var_threshold=5,
                     full_line_threshold=0.85, min_max_line_length=0),
    ReplaceMarksTransformer(marks_to_transform='„“', result_mark='"'),
    TooLongLinesTransformer(forbidden_delimiters='aábcčdďeéěfghiíjklmnňoópqrřsštťuúůvwxyýzž0123456789',
                            special_delimiters={'-': (r'[\s,\.](-)[\s]+[^(Kč)]', 1)},
                            too_long_line_treshold=200),
    RegexReplaceTransformer(pattern_to_transform=r'([^\n])[ ]*\n', result_pattern='\g<1>.\n'),
    RegexReplaceTransformer(pattern_to_transform=r'(([Nn]ázev|[Pp]opis)[^\n,.:"()]{5,})(\s[A-Z][^\n:]{10})',
                            result_pattern='\g<1>:\g<3>'),
    ReplaceMarksTransformer(marks_to_transform=[':'], result_mark=':.'),
    ReplaceMarksTransformer(marks_to_transform=['..'], result_mark='.'),
    RegexReplaceTransformer(pattern_to_transform=r'\([^\n()]*\)', result_pattern=''),
]

preprocessor = SubjectContextPreprocessor(transformers)

conllu_eval_transformers = [
    UdapiWordOccurrencePartSentenceFilter(keywords=['cena', 'hodnota', 'DPH']),
    UdapiWordOccurrencePartSentenceFilter(keywords=['příloha', 'dále', 'jen']),
    NonSubjectPartSentenceFilter(),
    EmptyBundlesFilter(),
]

conllu_eval_preprocessor = ConlluSubjectContextPreprocessor(transformers=conllu_eval_transformers)
conllu_evaluator = SubjectItemsEvaluationMachine(df_contracts, subj_context_preprocessor=preprocessor,
                                                 conllu_context_preprocessor=conllu_eval_preprocessor)
conllu_evaluator.preprocess()
conllu_evaluator.process()
conllu_evaluator.evaluate()

Wall time: 33.9 s


100.0

In [26]:
df_contracts = conllu_evaluator._df_contracts
df_contracts

,id,contr_name,text,subj_context,filtered_context,valid_score,filtered_context_ref,subj_context_decomposition,subj_items,subj_items_ref
0,71,e-zakazky/48a575ba-f8fe-47f2-bbb6-c6ddb26390ef...,=======================2017126-115451.pdf=====...,[PŘEDMĚT SMLOUVY\nZhotovitel se zavazuje k pro...,[PŘEDMĚT SMLOUVY.\nZhotovitel se zavazuje k pr...,1,PŘEDMĚT SMLOUVY.\nZhotovitel se zavazuje k pro...,[# newdoc\n# newpar\n# sent_id = 1\n# text = P...,[k provedení stavebních prací v rámci projektu...,k provedení stavebních prací v rámci projektu ...
1,72,e-zakazky/73b0be6b-6b49-4ab4-a6a0-508b83a305e1...,=======================Formulář - Informace o ...,[PŘEDMĚT DÍLA\n\n1.1. Předmětem díla je proved...,[PŘEDMĚT DÍLA.\n\nPředmětem díla je provedení ...,1,PŘEDMĚT DÍLA.\n\nPředmětem díla je provedení s...,[# newdoc\n# newpar\n# sent_id = 1\n# text = P...,[provedení stavby pro objednatele\ni geodetick...,provedení stavby pro objednatele\ni geodetické...
2,74,e-zakazky/c265fdf8-044f-44e5-b36e-df4ca53c6179...,=======================Dodatečné informace 01....,[Předmět smlouvy\n1. Zhotovitel se touto smlou...,[Předmět smlouvy.\nZhotovitel se touto smlouvo...,1,Předmět smlouvy.\nZhotovitel se touto smlouvou...,[# newdoc\n# newpar\n# sent_id = 1\n# text = P...,[dobudování kanalizace v obci Žádovice a výsta...,dobudování kanalizace v obci Žádovice a výstav...
3,61,eagri/mze/contract_display_6220,=======================01_výzva k podání nabíd...,[Předmět smlouvy\n\nPředmětem smlouvy je prove...,[Předmět smlouvy.\n\nPředmětem smlouvy je prov...,1,Předmět smlouvy.\n\nPředmětem smlouvy je prove...,[# newdoc\n# newpar\n# sent_id = 1\n# text = P...,"[stavebně technických průzkumů, zpracování pro...","stavebně technických průzkumů, zpracování proj..."
4,64,ezak/mpsv/P15V00000447,=======================AKRIS_příloha-01_Kvalif...,[Předmět veřejné zakázky a sjednaná cena ve sm...,[Předmět veřejné zakázky a sjednaná cena ve sm...,1,Předmět veřejné zakázky a sjednaná cena ve sml...,[# newdoc\n# newpar\n# sent_id = 1\n# text = P...,"[vytvoření, údržba, uživatelská podpora a rozv...","vytvoření, údržba, uživatelská podpora a rozvo..."
5,66,ezak/mpsv/P16V00000187,=======================Příloha č. 1 SOD_Projek...,"[Předmět smlouvy, předmět díla\n\nPředmětem té...","[Předmět smlouvy, předmět díla.\n\nPředmětem t...",1,"Předmět smlouvy, předmět díla.\n\nPředmětem té...",[# newdoc\n# newpar\n# sent_id = 1\n# text = P...,[],
6,67,ezak/mpsv/P16V00000240,=======================P1_Výkaz výměr (slepý)....,[Předmět Smlouvy\n2.1. Zhotovitel se zavazuje ...,[Předmět Smlouvy.\nZhotovitel se zavazuje prov...,1,Předmět Smlouvy.\nZhotovitel se zavazuje prové...,[# newdoc\n# newpar\n# sent_id = 1\n# text = P...,"[, ]",\n==========\n
7,103,nipez/mo/P18V00009525,=======================VZOR_smlouvy.doc=======...,[Předmět smlouvy\n\n2.1. \nPředmětem smlouvy j...,[Předmět smlouvy.\n\nPředmětem smlouvy je záva...,1,Předmět smlouvy.\n\nPředmětem smlouvy je závaz...,[# newdoc\n# newpar\n# sent_id = 1\n# text = P...,"[, ]",\n==========\n
8,207,nipez/mo/P18V00012708,=======================Oznámení o výběru dodav...,[Název veřejné zakázky: Sponkovače a spojovací...,[Název veřejné zakázky:. Sponkovače a spojovac...,1,Název veřejné zakázky:. Sponkovače a spojovací...,[# newdoc\n# newpar\n# sent_id = 1\n# text = N...,[],
9,397,nipez/mo/P18V00019354,=======================PSP s přílohami.zip====...,[Předmět smlouvy\n\n\n1. Poskytovatel se zavaz...,[Předmět smlouvy.\n\nPoskytovatel se zavazuje ...,1,Předmět smlouvy.\n\nPoskytovatel se zavazuje d...,[# newdoc\n# newpar\n# sent_id = 1\n# text = P...,[SW včetně dokumentace v českém nebo anglickém...,SW včetně dokumentace v českém nebo anglickém ...


### Playground

In [157]:
# 21 neextrahuje prvni kontext
# 24 extrakce vice ruznych enumerace
# 27 extrakce ?
# 37 castecna extrakce


In [34]:
df_contracts

,id,contr_name,text,subj_context,filtered_context,valid_score,filtered_context_ref
0,71,e-zakazky/48a575ba-f8fe-47f2-bbb6-c6ddb26390ef...,=======================2017126-115451.pdf=====...,[PŘEDMĚT SMLOUVY\nZhotovitel se zavazuje k pro...,[PŘEDMĚT SMLOUVY.\nZhotovitel se zavazuje k pr...,1.000000,PŘEDMĚT SMLOUVY.\nZhotovitel se zavazuje k pro...
1,72,e-zakazky/73b0be6b-6b49-4ab4-a6a0-508b83a305e1...,=======================Formulář - Informace o ...,[PŘEDMĚT DÍLA\n\n1.1. Předmětem díla je proved...,[PŘEDMĚT DÍLA.\n\nPředmětem díla je provedení ...,1.000000,PŘEDMĚT DÍLA.\n\nPředmětem díla je provedení s...
2,74,e-zakazky/c265fdf8-044f-44e5-b36e-df4ca53c6179...,=======================Dodatečné informace 01....,[Předmět smlouvy\n1. Zhotovitel se touto smlou...,[Předmět smlouvy.\nZhotovitel se touto smlouvo...,1.000000,Předmět smlouvy.\nZhotovitel se touto smlouvou...
3,61,eagri/mze/contract_display_6220,=======================01_výzva k podání nabíd...,[Předmět smlouvy\n\nPředmětem smlouvy je prove...,[Předmět smlouvy.\n\nPředmětem smlouvy je prov...,1.000000,Předmět smlouvy.\n\nPředmětem smlouvy je prove...
4,64,ezak/mpsv/P15V00000447,=======================AKRIS_příloha-01_Kvalif...,[Předmět veřejné zakázky a sjednaná cena ve sm...,[Předmět veřejné zakázky a sjednaná cena ve sm...,1.000000,Předmět veřejné zakázky a sjednaná cena ve sml...
5,66,ezak/mpsv/P16V00000187,=======================Příloha č. 1 SOD_Projek...,"[Předmět smlouvy, předmět díla\n\nPředmětem té...","[Předmět smlouvy, předmět díla.\n\nPředmětem t...",1.000000,"Předmět smlouvy, předmět díla.\n\nPředmětem té..."
6,67,ezak/mpsv/P16V00000240,=======================P1_Výkaz výměr (slepý)....,[Předmět Smlouvy\n2.1. Zhotovitel se zavazuje ...,[Předmět Smlouvy.\nZhotovitel se zavazuje prov...,1.000000,Předmět Smlouvy.\nZhotovitel se zavazuje prové...
7,103,nipez/mo/P18V00009525,=======================VZOR_smlouvy.doc=======...,[Předmět smlouvy\n\n2.1. \nPředmětem smlouvy j...,[Předmět smlouvy.\n\nPředmětem smlouvy je záva...,1.000000,Předmět smlouvy.\n\nPředmětem smlouvy je závaz...
8,207,nipez/mo/P18V00012708,=======================Oznámení o výběru dodav...,[Název veřejné zakázky: Sponkovače a spojovací...,[Název veřejné zakázky:. Sponkovače a spojovac...,1.000000,Název veřejné zakázky:. Sponkovače a spojovací...
9,397,nipez/mo/P18V00019354,=======================PSP s přílohami.zip====...,[Předmět smlouvy\n\n\n1. Poskytovatel se zavaz...,[Předmět smlouvy.\n\nPoskytovatel se zavazuje ...,1.000000,Předmět smlouvy.\n\nPoskytovatel se zavazuje d...


In [37]:
index = 36
for data in df_contracts.loc[index, 'filtered_context']:
    print(data)
    print('==========')

PŘEDMĚT VEŘEJNÉ ZAKÁZKY ......oooooeeoenooeooeoeeoeeoenvenoveoeevenné 4.
Vyhlášení zadávacího ŤÍZEníÍ............+r++r44444*2r4r44nFax44nF44n44H44nF44440F44044n4aK644F440F4HGene Hasta ate nné 4.
Klasifikace předmětu Veřejné zakázky u cuxuzisuerisssessiěiesáae vě úšsváoskni vání i dvináí dá cha o 4.
Vymezení předmětu veřejné zakázky .............eeeeeeeereeeereeeeeeoeeeeeeeeoeeedeedee tee otae tee eee one ote ate 4.
MÍSTO A DOBA PLNĚNÍ VEŘEJNÉ ZAKÁZKY........uoeeoroeeoroeeoooeeoeoeeeveee teen te neovo nanovo tenně 5.
Místo plnění veřejné zakázky ..............«++r.r+.444a0e4x44nr4ee44nFanete4e064n44nHaeAen one ona an ta Ane na nn tan 5.
Doba plnění veřejné Zakázky ..........e+orrrereeeeereeree eee oeeeneee n een EEE SERA KEREE EEE ERA RE RER RKK 5.
POŽADAVKY NA PROKÁZÁNÍ SPLŇENÍ KVALIFIKACE .......oooo oo eeooeoeeeeneeneeneeeevenne =.
Základní kvalifikační předpoklady zzz zssaossě zona ioronáoa neon voáa o ooovadsv da ovavčin coo sviněavíae 5.
Profesní kvalifikační předpoklady .........

In [30]:
print(df_contracts.loc[index, 'filtered_context_ref'])

Předmět Smlouvy.
Zhotovitel se zavazuje provést na svůj náklad a nebezpečí Dílo specifikované v příloze č. 1 této Smlouvy a dokončené Dílo předat Objednateli sjednaným způsobem a Objednatel se zavazuje dokončené Dílo převzít a zaplatit za ně sjednanou cenu.


==========Předmět Smlouvy.

Zhotovitel se zavazuje provést na svůj náklad a nebezpečí Dílo specifikované v příloze č. 1 této Smlouvy a dokončené Dílo předat Objednateli sjednaným způsobem a Objednatel se zavazuje dokončené Dílo převzít a zaplatit za ně sjednanou cenu.






In [40]:
for data in df_contracts.loc[index, 'subj_context']:
    print(data)
    print('==========')

Předmět smlouvy


1. Poskytovatel se zavazuje dodat SW včetně dokumentace v českém nebo anglickém jazyce v elektronické podobě tím, že dodá jen licenční klíče k SW dle přílohy č.1 Specifikace pořizovaného software.

2. Objednatel se zavazuje řádně dodaný SW převzít a zaplatit poskytovateli dohodnutou cenu podle čl. IV. této smlouvy. 


Předmět díla
1.
Předmětem smlouvy jsou stavební úpravy střechy a zastřešení vstupu 1.PP na objektu Pod Hradbami 662/9, 160 00 Praha 6.

2.
Rozsah prací je stanoven cenovou nabídkou v rozsahu položkového rozpočtu ze dne………………….. , který je přílohou č. 1 této smlouvy.
3.



Místem provádění díla je objekt na adrese:
Pod Hradbami 662/9, 160 00 Praha 6
Vymezení předmětu veřejné zakázky

Předmětem veřejné zakázky je pořízení komerčního software pro zajištění činností jednotlivých
složek resortu MO.

Klasifikace předmětu VZ: CPV

48000000-8 Balíky programů a informační systémy

Podrobné vymezení předmětu této veřejné zakázky je uvedeno v „Návrhu požadovaných s

In [38]:
save_subjects(df_contracts, 'filtered_context')

saving e-zakazky/48a575ba-f8fe-47f2-bbb6-c6ddb26390ef/P17V00000001
saving e-zakazky/73b0be6b-6b49-4ab4-a6a0-508b83a305e1/P12V00000002
saving e-zakazky/c265fdf8-044f-44e5-b36e-df4ca53c6179/P17V00000001
saving eagri/mze/contract_display_6220
saving ezak/mpsv/P15V00000447
saving ezak/mpsv/P16V00000187
saving ezak/mpsv/P16V00000240
saving nipez/mo/P18V00009525
saving nipez/mo/P18V00012708
saving nipez/mo/P18V00019354
saving nipez/mo/P18V00022561
saving nipez/mo/P19V00003599
saving nipez/mpsv/P18V00010416
saving nipez/mvcr/P18V00009631
saving nipez/mvcr/P18V00015153
saving nipez/mzv/P18V00000055
saving nipez/mzv/P18V00016340
saving nipez/mzv/P18V00017124
saving nipez/mzv/P18V00026759
saving nipez/mzv/P19V00001827
saving stavebnionline/121/7941
saving tenderarena/agrozet/102972
saving tenderarena/Comgate/201480
saving tenderarena/JihoceskyKraj/255714
saving tenderarena/Libereckykraj/219512
saving tenderarena/mo/171826
saving tenderarena/msp/85449
saving test_contracts/test_sub/contr_59
savin

In [246]:
class NonSubjectPartSentenceFilter(UdapiTransformer):

    def __init__(self,
                 target_dep_relations=['nsubj', 'obj', 'obl', 'obl:arg', 'nmod'],
                 target_verb_lemmas=['zavazovat', 'doda(t|ný)', 'zajistit', 'prov(edený|ést)',
                                     'zahrnovat', 'spočíva(t|jící)', 'rozumět'],
                 banned_upos_tags=['PUNCT', 'SYM', 'NUM', 'DET'],
                 banned_node_lemmas=['smlouva', 'předmět', 'uzavření', 'náklad', 'nebezpečí', 'specifikace',
                                     'dodavatel', 'závazek', 'dokumentace', 'rozsah', 'plnění', 'zakázka',
                                     'dohoda', 'poplatek', 'požádání', 'záměr', 'dodatek', 'podmínka', 'standard',
                                     'norma', 'kvalita', 'prohlášení', 'jiný'],
                 banned_submitter_lemmas=['objednatel', 'kupující', 'zadavate[lt]', 'projektant'],
                 banned_loctime_lemmas=['místo', 'doba'],
                 previous_banned_lemmas=['zbytný', 'pokud', 'dodatečný', 'dokumentace', 'nutný'],
                 preceding_banned_lemmas=['specifikovaný', 'povinný', 'oprávněný', 'možný', 'uvedený']):

        self._target_dep_relations = target_dep_relations
        self._target_verb_lemmas = target_verb_lemmas
        self._banned_upos_tags = banned_upos_tags
        self._banned_node_lemmas = banned_node_lemmas

        self._banned_submitter_lemmas = banned_submitter_lemmas
        self._banned_loctime_lemmas = banned_loctime_lemmas
        self._previous_banned_lemmas = previous_banned_lemmas
        self._preceding_banned_lemmas = preceding_banned_lemmas

        self._target_deprel_checker = NodeFinder(('deprel', self._target_dep_relations))
        self._banned_upos_checker = NodeFinder([('upos', self._banned_upos_tags), ('feats[Case]', 'Ins')])
        self._banned_lemmas_checker = NodeFinder(('lemma', self._banned_node_lemmas))

        self._banned_submitter_lemma_finder = PreviousNodeFinder(('lemma', self._banned_submitter_lemmas), 10, True)
        self._banned_loctime_lemma_finder = PreviousNodeFinder(('lemma', self._banned_loctime_lemmas), 10, True)
        self._previous_banned_lemma_finder = PreviousNodeFinder(('lemma', self._previous_banned_lemmas), 5)
        self._preceding_banned_lemma_finder = PrecedingNodeFinder(('lemma', self._preceding_banned_lemmas), 4)
        self._previous_target_verb_finder = PreviousNodeFinder([('lemma', self._target_verb_lemmas),
                                                                [('lemma', 'být'), ('deprel', ['cop','root']), ('feats[Polarity]', 'Pos')],
                                                               ], 5)

    def _get_candidate_nodes(self, tree):
        candidates = []
        for n in tree.descendants:

            if n.lemma == 'být' and n.deprel != 'cop':
                CNT.append((n.form, n.deprel, n.xpos))
#                 print(n.form + ': ' + n.xpos)

            if not self._target_deprel_checker.check_node_attributes(n):
                continue
                
            if self._banned_upos_checker.check_node_attributes(n):
                continue

            if self._banned_lemmas_checker.check_node_attributes(n):
                continue

            n2 = self._banned_submitter_lemma_finder.find(n)
            if n2 and (n2 == n or not PreviousNodeFinder(('lemma', 'pro'), 2).find(n2)):
                continue

            n2 = PreviousNodeFinder(('lemma', ['dílo']), 5, True).find(n)
            if n2 and (n2 == n or not (NextNodeFinder(('upos', 'DET'), 5).find(n2) or
                                       PreviousNodeFinder(('lemma', 'předmět'), 2).find(n2))):
                continue

            if self._banned_loctime_lemma_finder.find(n):
                continue

            if self._previous_banned_lemma_finder.find(n):
                continue

            if self._preceding_banned_lemma_finder.find(n):
                continue

            n2 = PreviousNodeFinder(('lemma', ['proveden(í|ý)']), 5).find(n)
            if n2 and PreviousNodeFinder([('lemma', 'předmět'), ('upos', 'DET')], 3).find(n2):
                candidates.append(n)
                continue

            if NodeFinder(('deprel', ['nmod'])).check_node_attributes(n):
                if PrecedingNodeFinder(('lemma', ['pořízení', 'uzavřen(í|ý)', 'zabezpečení'])).find(n):
                    candidates.append(n)
                continue

            if not self._previous_target_verb_finder.find(n):
                continue

            if NodeFinder(('deprel', ['obl', 'obl:arg'])).check_node_attributes(n):
                if not NodeFinder(('lemma', ['realizace'])).check_node_attributes(n):
                    if not PreviousNodeFinder(('lemma', ['předmět']), 5).find(n):
                        candidates.append(n)
                continue

            # deprel: nsubj, obj
            candidates.append(n)

        return candidates

    def _extend_candidates(self, candidates):
        # doplnim konjunkce
        extended_candidates = []
        i = 0
        while True:
            if not len(candidates) > i:
                break
            n = candidates[i]
            i += 1
            extended_candidates.append(n)
            # if not (extended_candidates and n.is_descendant_of(extended_candidates[-1])):
            #     extended_candidates.append(n)
            # else:
            #     n = extended_candidates[-1]
            if n.descendants and n.descendants[-1].next_node != n:
                n = n.descendants[-1]
            while True:
                n = NextNodeFinder(('deprel', ['conj', 'nsubj', 'obj']), 7).find(n)
                if not n or (len(candidates) > i and n.is_descendant_of(candidates[i])):
                    break
                extended_candidates.append(n)
        return extended_candidates

    def _get_all_node_candidates(self, node):
        tmp_candidates = []
        # ponecham jen slova za pripadnym nasledujicim slovesem
        n = DescendingNodeFinder([('upos', 'VERB'), ('deprel', 'cop')]).find(node)
        if n and abs(n.ord - node.ord) < 10 \
                and NodeFinder(('form', '(je|.{3,})')).check_node_attributes(n) \
                and not PreviousNodeFinder(('upos', 'DET'), 2).find(n):
            # abs(n2.ord - n.ord) < 10 kvuli bugu s nespravne tokenizovanzmi vetami
            # (je|.{3,}) kvuli bugu s identifikaci slovesa na nerelevantnich tokenech
            n = n.next_node
            while n and (n.is_descendant_of(node) or n == node):
                tmp_candidates.append(n)
                n = n.next_node
        else:
            tmp_candidates.append(node)
            tmp_candidates.extend(node.descendants)
            # odříznu příliš dlouhé řetězce
        keep_node_candidates = []
        for n in tmp_candidates:
            subj_node = PrecedingNodeFinder(('lemma', 'předmět')).find(n)
            if not (subj_node and subj_node.is_descendant_of(node)):
                if abs(n.ord - node.ord) > 10 and n.upos == 'PUNCT':
                    break
                keep_node_candidates.append(n)
        return keep_node_candidates

    def process(self, document):
        for i2, tree in enumerate(self._iter_sentences(document)):
            if not tree.children:
                continue

            candidates = self._get_candidate_nodes(tree)
            candidates = self._extend_candidates(candidates)

            keep_nodes = []
            for n in candidates:
                node_candidates = self._get_all_node_candidates(n)
                keep_nodes.extend(node_candidates)
            keep_nodes = [n for n in keep_nodes
                          if not NodeFinder(('lemma', ['smlouva', 'uzavření', 'předmět', 'plnění']))
                    .check_node_attributes(n)]

            to_remove = [n for n in tree.descendants if not n in keep_nodes]

            for n in keep_nodes:
                n.parent = tree
            for n in to_remove:
                n.remove()

            tree.text = tree.compute_text()
        return document


In [34]:
class StringFinder:
    
    _pattern = None
    _lower = None
    _show_prec_range = None
    _show_next_range = None
    
    def __init__(self, keyword, lower=True, show_range=30):
        self._pattern = keyword
        self._lower = lower
        self._show_prec_range = show_range[0] if isinstance(show_range, tuple) else show_range
        self._show_next_range = show_range[1] if isinstance(show_range, tuple) else show_range
    
    def process(self, text):
        occurrences = find_all_occurrences_in_string(self._pattern, text, self._lower)
        findings = {}
        for occ in occurrences:
            findings[occ] = text[max(0, occ-self._show_prec_range): min(occ+self._show_next_range, len(text))]
        return findings

class SearchEngine:
    
    _finders = None
    _findings = None

    def __init__(self, by_string=None, by_regex=None, by_conllu=None, match='exact', show_range=30):
        self._finders = []
        self._findings = {}
        for keyword in by_string:
            lower = False
            if match=='lower':
                lower = True
            self._finders.append(StringFinder(keyword, lower, show_range))
    
    def preprocess(self, data):
        data_collection = []
        if isinstance(data, str):
            data_collection.append(data)
        if isinstance(data, list) or \
           isinstance(data, pandas.core.series.Series):
            data_collection.extend([str(x) for x in data])
        return data_collection
    
    def process(self, data):
        texts = self.preprocess(data)
        for i, text in enumerate(texts):
            findings = {}
            for finder in self._finders:
                finding = finder.process(text)
                if finding:
                    findings[finder._pattern] = finding
            if findings:
                self._findings[i] = findings
        return self._findings

In [34]:
# validate_subjects(df_contracts, 'subj_context_decomposition')

In [35]:
df_sorted_contracts = df_contracts.copy()
df_sorted_contracts['context_len'] = df_sorted_contracts.subj_context.str.len()
df_sorted_contracts = df_sorted_contracts.sort_values('context_len').reset_index()
df_sorted_contracts

,index,id,contr_name,text,subj_context,filtered_context,subj_context_decomposition,valid_score,subj_context_decomposition_ref,sent_cnt_diff,subj_items,subj_items_ref,context_len
0,13,733,nipez/mvcr/P18V00009631,=======================017 Oznámení o výběru A...,veřejné zakázce\n\n„Náboje 308 Winchester SWISS“,"veřejné zakázce.\n\n""Náboje 308 Winchester SWI...",# newdoc\n# newpar\n# sent_id = 1\n# text = ve...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = ve...,0,,,46
1,8,207,nipez/mo/P18V00012708,=======================Oznámení o výběru dodav...,Název veřejné zakázky: Sponkovače a spojovací ...,Název veřejné zakázky:. Sponkovače a spojovací...,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0,,,54
2,15,771,nipez/mzv/P18V00000055,=======================Plovoucí vinylová podla...,Název veřejné zakázky:\n\tPlovoucí vinylová po...,Název veřejné zakázky:.\n\tPlovoucí vinylová p...,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0,,,67
3,14,755,nipez/mvcr/P18V00015153,=======================#KS DNS ICT 11-2018-III...,Předmět plnění\n1. Předmětem této smlouvy je d...,Předmět plnění.\nPředmětem této smlouvy je dod...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0,dodávka zboží vymezeného,dodávka zboží vymezeného,229
4,6,67,ezak/mpsv/P16V00000240,=======================P1_Výkaz výměr (slepý)....,Předmět Smlouvy\n2.1. Zhotovitel se zavazuje p...,Předmět Smlouvy.\nZhotovitel se zavazuje prové...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0,,,262
5,9,397,nipez/mo/P18V00019354,=======================PSP s přílohami.zip====...,Předmět smlouvy\n\n\n1. Poskytovatel se zavazu...,Předmět smlouvy.\n\nPoskytovatel se zavazuje d...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0,SW včetně dokumentace v českém nebo anglickém ...,SW včetně dokumentace v českém nebo anglickém ...,337
6,39,849,vhodne-uverejneni/zakladni-skola-velke-hamry-s...,=======================deklerace k dodatku č. ...,Název veřejné zakázky:\nZadavatel zakázky:\n\n...,Název veřejné zakázky:.\nZadavatel zakázky:.\n...,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = Ná...,0,,,339
7,31,831,vhodne-uverejneni/buly-arena-aquapark-kravare-...,=======================xx Písemná zpráva zadav...,předmět veřejné zakázky a cena sjednaná ve sml...,předmět veřejné zakázky a cena sjednaná ve sml...,# newdoc\n# newpar\n# sent_id = 1\n# text = př...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = př...,0,o dodávce elektrické energie,o dodávce elektrické energie,456
8,0,71,e-zakazky/48a575ba-f8fe-47f2-bbb6-c6ddb26390ef...,=======================2017126-115451.pdf=====...,PŘEDMĚT SMLOUVY\nZhotovitel se zavazuje k prov...,PŘEDMĚT SMLOUVY.\nZhotovitel se zavazuje k pro...,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,0,k provedení stavebních prací v rámci projektu ...,k provedení stavebních prací v rámci projektu ...,610
9,16,775,nipez/mzv/P18V00016340,=======================VZ-096.docx============...,Předmět veřejné zakázky\nStručný popis předmět...,Předmět veřejné zakázky.\nStručný popis předmě...,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,1,# newdoc\n# newpar\n# sent_id = 1\n# text = Př...,0,,,627


In [36]:
class SubjectItemsEvaluationMachineDBG:
    
    def __init__(self, df_contracts,\
                 subj_context_preprocessor=SubjectContextPreprocessor(),\
                 conllu_context_preprocessor=ConlluSubjectContextPreprocessor()):
        self._df_contracts = df_contracts
        self._subj_context_extractor = subj_context_preprocessor
        self._conllu_context_extractor = conllu_context_preprocessor
        self._log_file = '../../test-data/debug.log'
        self._old_stdout = sys.stdout

        
    def preprocess(self):
        sys.stdout = open(self._log_file, 'w', encoding='utf8')
        self._df_contracts['filtered_context'] = self._df_contracts['subj_context'].apply(\
            lambda text: self._subj_context_extractor.process(text))
        self._df_contracts['subj_context_decomposition'] = self._df_contracts['filtered_context'].apply(\
            lambda text: udp_pipeline.process(text))
        self._df_contracts['subj_items'] = self._df_contracts['subj_context_decomposition'].apply(\
            lambda text: str(self._conllu_context_extractor.process(create_conllu_document(text))))
        sys.stdout.close()
        sys.stdout = self._old_stdout
        return self._df_contracts
    
    def process(self):
        self._df_contracts = validate_subjects(self._df_contracts, 'subj_items')
        return self._df_contracts

    def evaluate(self):
        return self._df_contracts['valid_score'].mean()*100

In [37]:
class NonSubjectPartSentenceFilterDBG(NonSubjectPartSentenceFilter):
    
    def _log(self, mark, node):
        msg = '<'+mark+'>'+ node.lemma if node is not None else ''
        print(msg)

    def _get_candidate_nodes(self, tree):
        candidates = []
        for n in tree.descendants:
            self._log('-',n)

            if not NodeFinder(('deprel', ['nsubj', 'obj', 'obl', 'nmod'])).check_node_attributes(n):
                self._log('21',n)
                continue
            self._log('51',n)

            if NodeFinder([('upos', ['PUNCT', 'SYM', 'NUM', 'DET']), \
                           ('feats[Case]', 'Ins')]).check_node_attributes(n):
                self._log('01',n)
                continue
            self._log('31',n)

            if NodeFinder([[('lemma', ['smlouva', 'předmět', 'uzavření', 'náklad', 'nebezpečí', 'specifikace',
                                      'dodavatel', 'závazek',
                                     'dokumentace', 'rozsah', 'plnění', 
                                      'zakázka','dohoda', 
                                       'poplatek','požádání','záměr', 'dodatek',]),\
                            ]]).check_node_attributes(n):
                self._log('055',n)
                continue
            self._log('355', n)
            
            n2 = PreviousNodeFinder(('lemma', ['objednatel', 'kupující', 'zadavate[lt]', 'projektant']), 10, True).find(n)
            if n2 and (n2 == n or not PreviousNodeFinder(('lemma', 'pro'), 2).find(n2)):
                self._log('03',n)
                continue
            self._log('33',n)

            n2 = PreviousNodeFinder(('lemma', ['dílo']), 5, True).find(n)
            if n2 and (n2 == n or not (NextNodeFinder(('upos', 'DET'), 5).find(n2) or \
                                       PreviousNodeFinder(('lemma', 'předmět'), 2).find(n2))):
                self._log('02',n)
                continue
            self._log('32',n)

            if PreviousNodeFinder(('lemma', ['místo', 'doba']), 10, True).find(n):
                self._log('04',n)
                continue
            self._log('34',n)

            if PreviousNodeFinder(('lemma', ['zbytný', 'pokud', 'dodatečný', 'dokumentace']), 5).find(n):
                self._log('057',n)
                continue
            self._log('357', n)
            
            if PrecedingNodeFinder(('lemma', ['specifikovaný', 'povinný', 'oprávněný', 'možný', 'uvedený',
                                             ]), 4).find(n):
                self._log('058',n)
                continue
            self._log('358', n)
                

            if NodeFinder(('deprel', ['nmod'])).check_node_attributes(n):
                self._log('17',n)
                if PrecedingNodeFinder(('lemma', ['pořízení', 'uzavřen(í|ý)'])).find(n):
                    self._log('18',n)
                    candidates.append(n)
                    continue
            self._log('46',n)

            n2 = PreviousNodeFinder(('lemma', ['proveden(í|ý)']), 5).find(n)
            if n2 and PreviousNodeFinder([('lemma', 'předmět'), ('upos', 'DET')], 3).find(n2):
                self._log('20',n)
                candidates.append(n)
                continue
            self._log('50', n)
                
            if not PreviousNodeFinder([('lemma', ['zavazovat', 'doda(t|ný)', 'zajistit', 'prov(edený|ést)',
                                                  'zahrnovat', 'spočíva(t|jící)', 'rozumět']),
                                        [('deprel', ['cop']), ('feats[Polarity]', 'Pos')]], 5).find(n):
                self._log('056', n)
                continue
            self._log('356',n)

            if NodeFinder(('deprel', ['obl'])).check_node_attributes(n) and \
                    not NodeFinder(('lemma', ['realizace'])).check_node_attributes(n):
                self._log('10',n)
                if not PreviousNodeFinder(('lemma', ['předmět']), 5).find(n):
                    self._log('11',n)
                    candidates.append(n)
                    continue
            self._log('40',n)            

            if NodeFinder(('deprel', ['nsubj'])).check_node_attributes(n):
                self._log('06',n)
                candidates.append(n)
                continue
            self._log('36',n)

            if NodeFinder(('deprel', ['obj'])).check_node_attributes(n):
                self._log('15',n)
                candidates.append(n)
                continue
            self._log('45',n)                       

        return candidates
    
    def _extend_candidates(self, candidates):
        # doplnim konjunkce
        extended_candidates = []
        i = 0
        while True:
            if not len(candidates) > i:
                break
            n = candidates[i]
            i += 1
            if not (extended_candidates and n.is_descendant_of(extended_candidates[-1])):
                extended_candidates.append(n)
            else:
                n = extended_candidates[-1]
            if n.descendants and n.descendants[-1].next_node != n:
                n = n.descendants[-1]
            while True:
                n = NextNodeFinder(('deprel', ['conj', 'nsubj', 'obj']), 7).find(n)
                if not n or (len(candidates) > i and n.is_descendant_of(candidates[i])):
                    break
                extended_candidates.append(n)
        return extended_candidates

    def _get_all_node_candidates(self, node):
        tmp_candidates = []
        # ponecham jen slova za pripadnym nasledujicim slovesem
        n = DescendingNodeFinder([('upos', 'VERB'), ('deprel', 'cop')]).find(node)
        if n and abs(n.ord - node.ord) < 10 \
                and NodeFinder(('form', '(je|.{3,})')).check_node_attributes(n) \
                and not PreviousNodeFinder(('upos', 'DET'), 2).find(n):
            # abs(n2.ord - n.ord) < 10 kvuli bugu s nespravne tokenizovanzmi vetami
            # (je|.{3,}) kvuli bugu s identifikaci slovesa na nerelevantnich tokenech
            n = n.next_node
            while n and (n.is_descendant_of(node) or n == node):
                tmp_candidates.append(n)
                n = n.next_node
        else:
            tmp_candidates.append(node)
            tmp_candidates.extend(node.descendants)
            # odříznu příliš dlouhé řetězce
        keep_node_candidates = []
        for n in tmp_candidates:
            subj_node = PrecedingNodeFinder(('lemma', 'předmět')).find(n)
            if not (subj_node and subj_node.is_descendant_of(node)):
                if abs(n.ord - node.ord) > 10 and n.upos == 'PUNCT':
                    break
                keep_node_candidates.append(n)
        return keep_node_candidates

    def process(self, document):
        for i2, tree in enumerate(self._iter_sentences(document)):
            if not tree.children:
                continue

            candidates = self._get_candidate_nodes(tree)
            candidates = self._extend_candidates(candidates)

            keep_nodes = []
            for n in candidates:
                node_candidates = self._get_all_node_candidates(n)
                keep_nodes.extend(node_candidates)
            keep_nodes = [n for n in keep_nodes
                          if not NodeFinder(('lemma', ['smlouva', 'uzavření', 'předmět', 'plnění']))
                    .check_node_attributes(n)]

            to_remove = [n for n in tree.descendants if not n in keep_nodes]

            for n in keep_nodes:
                n.parent = tree
            for n in to_remove:
                n.remove()

            tree.text = tree.compute_text()
        return document


In [38]:
%%time

preprocessor = SubjectContextPreprocessor()

conllu_eval_transformers = [
    UdapiWordOccurrencePartSentenceFilter(keywords=['cena', 'hodnota', 'DPH']),
#     SubmitterPartSentenceFilter(submitter_keywords=['objednatel', 'zadavatel', 'kupující'],
#                                 suplier_keywords=['dodavatel', 'prodávající', 'zhotovitel', 'uchazeč']),
    UdapiWordOccurrencePartSentenceFilter(keywords=['příloha', 'dále', 'jen']),
#     UdapiWordOccurrencePartSentenceFilter(keywords=['náklad', 'nebezpečí','odpovědnost']),
#     UdapiWordOccurrencePartSentenceFilter(keywords=['místo', 'doba']),
    NonSubjectPartSentenceFilter(
                target_dep_relations = ['nsubj', 'obj', 'obl', 'nmod'],
                target_verb_lemmas = ['zavazovat', 'doda(t|ný)', 'zajistit', 'prov(edený|ést)',
                                      'zahrnovat', 'spočíva(t|jící)', 'rozumět'],
                banned_upos_tags = ['PUNCT', 'SYM', 'NUM', 'DET'],
                banned_node_lemmas = ['smlouva', 'předmět', 'uzavření', 'náklad', 'nebezpečí', 'specifikace',
                                    'dodavatel', 'závazek', 'dokumentace', 'rozsah', 'plnění', 'zakázka',
                                    'dohoda', 'poplatek', 'požádání', 'záměr', 'dodatek'],
                banned_submitter_lemmas = ['objednatel', 'kupující', 'zadavate[lt]', 'projektant'],
                banned_loctime_lemmas = ['místo', 'doba'],
                previous_banned_lemmas = ['zbytný', 'pokud', 'dodatečný', 'dokumentace'], 
                preceding_banned_lemmas = ['specifikovaný', 'povinný', 'oprávněný', 'možný', 'uvedený']),
    EmptyBundlesFilter(),
]
df_slice = df_sorted_contracts[30:].copy()
conllu_eval_preprocessor = ConlluSubjectContextPreprocessor(transformers=conllu_eval_transformers)
conllu_evaluator = SubjectItemsEvaluationMachine(df_sorted_contracts, subj_context_preprocessor=preprocessor, conllu_context_preprocessor=conllu_eval_preprocessor)
conllu_evaluator.preprocess()
conllu_evaluator.process()
conllu_evaluator.evaluate()

Wall time: 31.8 s


100.0

In [755]:
df_conllu_contracts = conllu_evaluator._df_contracts
df_conllu_contracts[df_conllu_contracts.valid_score < 1]

,index,id,contr_name,text,subj_context,filtered_context,subj_context_decomposition,valid_score,subj_context_decomposition_ref,sent_cnt_diff,context_len,subj_items,subj_items_ref
37,1,72,e-zakazky/73b0be6b-6b49-4ab4-a6a0-508b83a305e1...,=======================Formulář - Informace o ...,PŘEDMĚT DÍLA\n\n1.1. Předmětem díla je provede...,PŘEDMĚT DÍLA.\n\nPředmětem díla je provedení s...,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,0.611111,# newdoc\n# newpar\n# sent_id = 1\n# text = PŘ...,0,8293,provedení stavby pro objednatele\ni geodetické...,provedení stavby pro objednatele\ni geodetické...


In [775]:
for neco in df_conllu_contracts.loc[37, ['subj_items', 'subj_items_ref']]:
    print('----------------')
    print(neco)

----------------
provedení stavby pro objednatele
i geodetické zaměření dokončeného díla
realizace výše
----------------
provedení stavby pro objednatele
i geodetické zaměření dokončeného díla
realizace výše


In [628]:
default_stdout = sys.stdout

In [772]:
sys.stdout = default_stdout

In [312]:
transformed_texts = [preprocessor.process(t) for t in df_sorted_contracts.subj_context]

In [313]:
se = SearchEngine(by_string=['uzavření'], match='exact', show_range=(50,50))
se.process(transformed_texts)

{7: {'uzavření': {285: 'organizace « Předmět Předmětem veřejné zakázky je uzavření smlouvy o dodávce elektrické energie. \n\n'}},
 11: {'uzavření': {106: 'akázky a jeho specifikace.\n\nZáměrem zadavatele je uzavření smlouvy, na základě které vybraný uchazeč',
   212: 'ou odpovědnost a náklady dodá zadavateli po jejím uzavření předmět plnění této zakázky (zboží), kter'}},
 17: {'uzavření': {532: 'zahájení plnění předmětu veřejné zakázky ihned po uzavření smlouvy a ukončení dodávky nejpozději do ',
   596: 'vy a ukončení dodávky nejpozději do 60 dnů od dne uzavření smlouvy. Místem plnění je areál firmy ZDA'}},
 20: {'uzavření': {98: 'ejné zakázky je ve smyslu ustanovení $ 134 zákona uzavření rámcové dohody s jedním dodavatelem na do',
   529: 'emné výzvy k poskytnutí plnění, jež je návrhem na uzavření smlouvy, a písemného potvrzení této výzvy'}},
 30: {'uzavření': {986: '. Zhotovitel prohlašuje, že nejpozději k okamžiku uzavření této smlouvy se s přílohou č. 1 a č. 2 té',
   1648: 'vést 

In [41]:
%%time
# 26, 29
conllu_transformers = [
    UdapiWordOccurrencePartSentenceFilter(keywords=['cena', 'hodnota', 'DPH']),
#     SubmitterPartSentenceFilter(submitter_keywords=['objednatel', 'zadavatel', 'kupující'],
#                                 suplier_keywords=['dodavatel', 'prodávající', 'zhotovitel', 'uchazeč']),
    UdapiWordOccurrencePartSentenceFilter(keywords=['příloha', 'dále', 'jen']),
#     UdapiWordOccurrencePartSentenceFilter(keywords=['náklad', 'nebezpečí','odpovědnost']),
#     UdapiWordOccurrencePartSentenceFilter(keywords=['místo', 'doba']),
]
conllu_preprocessor = ConlluSubjectContextPreprocessor(transformers=conllu_transformers)

conllu_transformers2 = [
    NonSubjectPartSentenceFilter(),
    EmptyBundlesFilter(),
]
conllu_preprocessor2 = ConlluSubjectContextPreprocessor(transformers=conllu_transformers2)

index = 24
text = df_sorted_contracts.subj_context.iloc[index]
print(text)
print('------------------')
transformed_text = preprocessor.process(text)
print(transformed_text)
print('------------------')
decomposition = udp_pipeline.process(transformed_text)
doc = Document()
doc.from_conllu_string(decomposition)
doc = conllu_preprocessor.process(doc)
display(doc)
print('------------------')
doc = conllu_preprocessor2.process(doc)
display(doc)

Předmět smlouvy


1. Název díla: Rekonstrukce vytápění v objektu SPV Pod Kamenem 179 – vybrané části.

2. Specifikace díla:


Rozsah díla je dán Soupisem stavebních prací, dodávek a služeb s výkazem výměr, který byl součástí zadávací dokumentace veřejné zakázky malého rozsahu „Rekonstrukce vytápění v objektu SPV Pod Kamenem 179“, který tvoří přílohu č. 1 této smlouvy.

V rámci realizace budou provedeny zejména práce na rekonstrukci chodníku v následujícím rozsahu:

		PS01

		 

		STL plynová přípojka

		 

		 



		PS01

		57

		Kryty štěrkových a živičných pozemních komunikací a zpevněných ploch

		 

		 



		PS01

		573211111R00

		Postřik živičný spojovací z asfaltu 0,5-0,7 kg/m2

		m2

		450,66



		

		

		429,2*1,05

		

		450,66



		PS01

		572713111R00

		Vyrovnání povrch krytů živ. směsí, koberec otevřený

		t

		2,58



		

		

		429,2*0,05*0,05*2,4

		

		2,58



		PS01

		577131311R00

		Beton asfaltový ACO 8 CH, š. do 3 m, tl. 4 cm

		m2

		450,66



		

		

		429,2*1,05

Předmět smlouvy.
Název díla:.
Rekonstrukce vytápění v objektu SPV Pod Kamenem 179 – vybrané části.
Specifikace díla:.
Rozsah díla je dán Soupisem stavebních prací, dodávek a služeb s výkazem výměr, který byl součástí zadávací dokumentace veřejné zakázky malého rozsahu "Rekonstrukce vytápění v objektu SPV Pod Kamenem 179", který tvoří .
V rámci realizace budou provedeny zejména práce na rekonstrukci chodníku v následujícím rozsahu:.
STL plynová přípojka.
Kryty štěrkových a živičných pozemních komunikací a zpevněných ploch.
Postřik živičný spojovací z asfaltu 0,5-0,7 kg/m2.
Vyrovnání povrch krytů živ. směsí, koberec otevřený.
Beton asfaltový ACO 8 CH, š. do 3 m, tl. 4 cm.
Přesun hmot, oprava komunikací, kryt živič. a bet.
Přesun hmot, oprava komunikací, příplatek do 5 km.
Různé dokončovací konstrukce a práce inženýrských staveb.
Očištění povrchu krytu saponátovým roztokem.
Místo plnění:.
Ul. Pod Kamenem, Český Krumlov.
<ITEM>STL plynová přípojka<ITEM/> <ITEM>Kryty štěrkových a živičných 

------------------


zejména práce na rekonstrukci chodníku

Wall time: 639 ms


In [616]:
tmp_doc = Document()
tmp_doc.from_conllu_string(decomposition)
tmp_doc

PŘEDMĚT SMLOUVY.
Zhotovitel se zavazuje k provedení stavebních prací v rámci projektu "Stezka pro cyklisty - k. ú. Starý Mateřov" .
Objednatel se zavazuje předmět smlouvy převzít bez vad a nedodělků v době předání a zaplatit za ně zhotoviteli cenu podle smlouvy a podmínek dohodnutých ve smlouvě.
Předmětem plnění je Stezka pro cyklisty — k. ú. Starý Mateřov v souladu s podmínkami provádění díla stanovenými projektovou dokumentací.
Dílo bude provedeno v rozsahu projektové dokumentace, zadávací dokumentace a položkového rozpočtu — položkového výkazu výměr oceněného uchazečem.
<CONTRACT_NAME>Stezka pro cyklisty - k. ú. Starý Mateřov<CONTRACT_NAME/>

In [227]:
tree = tmp_doc.bundles[1].trees[0]
tree.print_subtree()

# sent_id = 2
# text = Předmětem plnění je dodávka a nákup zemědělských strojů včetně příslušenství a služeb s touto činností souvisejících , Prodávající se zavazuje kupujícímu dodat následující zboží:. kus Nakladač kolový kloubový JCB 436 c HT AGRI HL.
─┮
 ╰─┮ Předmětem NOUN root
   ┡─╼ plnění NOUN nmod
   ┡─╼ je AUX cop
   ┡─┮ dodávka NOUN nsubj
   │ │ ╭─╼ a CCONJ cc
   │ ┡─┶ nákup NOUN conj
   │ │ ╭─╼ zemědělských ADJ amod
   │ ┡─┾ strojů NOUN nmod
   │ │ │ ╭─╼ včetně ADP case
   │ │ ╰─┾ příslušenství NOUN nmod
   │ │   │ ╭─╼ a CCONJ cc
   │ │   ╰─┶ služeb NOUN conj
   │ │   ╭─╼ s ADP case
   │ │   ┢─╼ touto DET det
   │ │ ╭─┶ činností NOUN obl:arg
   │ ┡─┶ souvisejících ADJ amod
   │ │ ╭─╼ , PUNCT punct
   │ │ ┢─╼ Prodávající ADJ nsubj:pass
   │ │ ┢─╼ se PRON expl:pass
   │ ╰─┾ zavazuje VERB appos
   │   ╰─┮ kupujícímu ADJ obj
   │     ╰─┮ dodat VERB advcl
   │       │ ╭─╼ následující ADJ amod
   │       ┡─┶ zboží NOUN obj
   │       │ ╭─┮ : PUNCT punct
   │       │ │ ╰─╼ . PUNCT p

In [532]:
tmp_doc.store_conllu('../../test-data/test_src/conllu_temp_'+str(index)+'.txt')

In [41]:
tmp_doc.store_conllu('../../test-data/test_src/conllu_tmp.txt')

In [193]:
def save_connlu_to_file(conllu_data, path='../../test-data/test_src/conllu_temp.txt'):
    if isinstance(conllu_data, list):
        conllu_data = '\n'.join([sentence.serialize() for sentence in conllu_data])
    if isinstance(conllu_data, conllu.models.TokenList):
        conllu_data = conllu_data.serialize()
    with open(path, 'w', encoding='utf-8') as f:
        f.write(conllu_data)

def save_text_as_connlu(text, udp_pipeline, path='../../test-data/test_src/conllu_temp.txt'):
    decomposition = udp_pipeline.process(text)
    save_connlu_to_file(decomposition, path)

In [51]:
save_connlu_to_file(doc, path='../../test-data/conllu_temp_'+str(index)+'.txt')

TypeError: write() argument must be str, not Document